![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=Earth+Engine+To+BigQuery.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/Earth%20Engine%20To%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FEarth%2520Engine%2520To%2520BigQuery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/Earth%20Engine%20To%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/Earth%20Engine%20To%20BigQuery.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Earth Engine To BigQuery


**Google Earth Engine**

[Google Earth Engine](https://earthengine.google.com/) is a giant catelog of satellite imagery and geospatial datasets. The best part is that it has currated sets of prepared data - think pixel level detection with masking for clouds.  The data can be eplored and analyzed with the [Earth Engine API](https://developers.google.com/earth-engine/#api) which includes a [Python client](https://developers.google.com/earth-engine/guides/python_install).

Some Datasets of Interest:
- [Dynamic World V1](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)
    - 10m resolution with high frequency
    - A set of predictions based on analytics of Sentinel-2 L1C (see next) where CLOUDY_PIXEL_PERCENTAGE <= 35%
    - Bands for: water, trees, grass, flooded_vegetation, crops, shrub_and_scrub, built, bare, snow_and_ice, label
    - All are probabilities (except label) and add to 1.0. Label has index of the band with highest probability 0-8.
    - `ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')`
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/sentinel-2)
    - MultiSpectral Imaging since 6/23/2015
    - Capturing reflectance of [surface (L2A)](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) and [top-of-atmosphere (L1C)](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED)
    - `ee.ImageCollection('COPERNICUS/S2')` or `ee.ImageCollection('COPERNICUS/S2_HARMONIZED`

---
## Setup

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [15]:
SERIES = 'applied-geospatial'
EXPERIMENT = 'ee-to-bq'

### Enable Earth Engine API

[Console Linl](https://console.cloud.google.com/apis/library/earthengine.googleapis.com)

In [2]:
!gcloud services enable earthengine.googleapis.com

Operation "operations/acat.p2-1026793852137-5d047d6a-fd0e-4a8b-b871-d28d5d6f0205" finished successfully.


In [14]:
!gcloud services list --enabled --filter="BigQuery OR EarthEngine"

NAME                                TITLE
bigquery.googleapis.com             BigQuery API
bigqueryconnection.googleapis.com   BigQuery Connection API
bigquerymigration.googleapis.com    BigQuery Migration API
bigqueryreservation.googleapis.com  BigQuery Reservation API
bigquerystorage.googleapis.com      BigQuery Storage API
cloudtrace.googleapis.com           Cloud Trace API
earthengine.googleapis.com          Google Earth Engine API


### Configure Earth Engine Access

Configure a Google Cloud Project:
- [Details](https://developers.google.com/earth-engine/guides/access#a-role-in-a-cloud-project)
    - Enabled API for Earth Engine, section above
    - Register this cloud project with Earth Engine: [link](https://code.earthengine.google.com/register)